In [14]:
import pandas as pd
import numpy as np

# Data Cleaning & Manipulation
With Pandas, we can do more than just select data that is already there. 
We can add new columns to our datasets, apply functions, iterate thorugh each row in the dataframe, and more.

This is where we move from "pandas for exploring our data" to "pandas for getting our data ready to feed into models".

In [15]:
wine = pd.read_csv('data/wine_reviews/winemag-data_first150k.csv', index_col=0)
energy = pd.read_csv('data/energy/PJM_Load_hourly.csv', parse_dates=True, index_col=0)

### Dealing with Null Values:

In [16]:
wine.isnull().sum() * 100 / len(wine)

country         0.003313
description     0.000000
designation    30.302127
points          0.000000
price           9.073743
province        0.003313
region_1       16.603724
region_2       59.615053
variety         0.000000
winery          0.000000
dtype: float64

**Question:** why did we use `.any()` twice when asking this question?

What happens if we just use `.any()` once?

Now, we need to deal with the null values in the wine table.
How we deal with them will depend on the data type of the column in which null values are found, and how we hope to use the data.

If a wine doesn't have a region listed, we can still get a lot of information about it from things like its country. But, if a wine doesn't have a price listed, it's difficult to get much information out of it. 

So, if a wine has a null value for either of its regions, we will set it to be "unknown". But, if it has a null price, we will drop the row.

The 5 rows where a NaN value existed for country have been dropped.

Now we'll do the same for rows with a null price.

Wow! We lost a lot of rows getting rid of the null price rows.
Can you think of another way we could deal with null prices that doesn't mean losing almost 1/10 of our data? What are the pros and cons to using these other methods?

Now, for the other columns in which there are nulls, we can use `df.fillna()` to populate the null values with a value of our choosing. 

This is where we have to make some decisions. Improperly setting variables can have unintended consequences on any analysis we plan to do later using this data.

The only columns where we still have nulls are regions and designations.
To avoid having to drop out more rows, we will use "fillna()".

We now have no null values in our dataset!

In [17]:
# visualizing the df to see rows where "unknown" has been inserted


### Creating new columns from existing ones
In pandas, it's easy to make a new column from existing ones. 

#### Using a function + apply to make a new column:
Let's say we want to make a new column with an ID we can use to identify the records.
We can use a helper function + the apply method in pandas to apply it to all rows.
Apply looks through each row or column of a dataframe (depending on Axis set).

We can use the same approach to make a column for "great and cheap" wines, defined here as $50 or less and with a score >95.

We can then use that boolean value to sort on the wines.

**Exercise:** Make a new column in the energy dataset, showing the energy in time T-1 for each time T. Hint: there is a built-in method in Pandas that can help with this, no complicated functions needed :)

**Exercise:** Re-read the CSV for the wine data, and write a function to take care of all the nan values. Bonus points if it includes a test.

## Updates: Pandas 1.0
Pandas had already been a well-tested package used in development environments for quite some time. The official move to version 1.0 mainly just included some format fixes and 
- Dedicated types for String, boolean and NaN (before, just called "object")
- New documentation


In [18]:
wine

,country,description,designation,points,price,province,region_1,region_2,variety,winery
0,US,This tremendous 100% varietal wine hails from ...,Martha's Vineyard,96,235.0,California,Napa Valley,Napa,Cabernet Sauvignon,Heitz
1,Spain,"Ripe aromas of fig, blackberry and cassis are ...",Carodorum Selección Especial Reserva,96,110.0,Northern Spain,Toro,NaN,Tinta de Toro,Bodega Carmen Rodríguez
2,US,Mac Watson honors the memory of a wine once ma...,Special Selected Late Harvest,96,90.0,California,Knights Valley,Sonoma,Sauvignon Blanc,Macauley
3,US,"This spent 20 months in 30% new French oak, an...",Reserve,96,65.0,Oregon,Willamette Valley,Willamette Valley,Pinot Noir,Ponzi
4,France,"This is the top wine from La Bégude, named aft...",La Brûlade,95,66.0,Provence,Bandol,NaN,Provence red blend,Domaine de la Bégude
...,...,...,...,...,...,...,...,...,...,...
150925,Italy,Many people feel Fiano represents southern Ita...,NaN,91,20.0,Southern Italy,Fiano di Avellino,NaN,White Blend,Feudi di San Gregorio
150926,France,"Offers an intriguing nose with ginger, lime an...",Cuvée Prestige,91,27.0,Champagne,Champagne,NaN,Champagne Blend,H.Germain
150927,Italy,This classic example comes from a cru vineyard...,Terre di Dora,91,20.0,Southern Italy,Fiano di Avellino,NaN,White Blend,Terredora
150928,France,"A perfect salmon shade, with scents of peaches...",Grand Brut Rosé,90,52.0,Champagne,Champagne,NaN,Champagne Blend,Gosset


### create a column which equals 1 if the wine costs less than 50 and has more points then 90


In [26]:
#boolean mask style

wine1 = wine.copy()

bool_50 = wine1.loc[:, "price"] <= 50
bool_95 = wine1.loc[:, "points"] >= 95

bool_total = bool_50 & bool_95
wine1.loc[:, "cheap_and_great"] = bool_total.astype(int)

In [27]:
#pd. datafrme iterrows

wine2 = wine.copy()

wine2.loc[:, "cheap_and_great"] = 0
for index, row in wine2.iterrows():
    if (row["price"] <= 50 and row["points"] >= 95):
        wine2.loc[index, "cheap_and_great"] = 1 

In [31]:
#use a apply lambda function

wine3 = wine.copy

def helper(x: pd.Series):
    if x["price"] <= 50 and x["points"] >= 95:
        return 1
    else:
        return 0

wine.apply(lambda x: x["price"] <= 50 and x["points"] >= 95, axis=1).sum()

409